step1.
quick computing in edge,provide reliability
step2.
feature importance distributing,providing reliability
step3.
traditional data --> compression --> ML --> Acc
BE data(by feature importance) --> compression --> ML --> Acc

In [149]:
import jupyter_Importer
from MLModels_FeatureImportance import featureImportance_linearRegression
from MLModels_FeatureImportance import featureImportance_permutation
from MLModels_FeatureImportance import featureImportance_DTree

import pandas as pd
import numpy as np
import math
from sklearn import preprocessing
from sklearn.cluster import KMeans
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split


In [150]:
dataset = pd.read_csv("Occupancy.csv", names = ['x' + str(i) for i in range(1, 7)] + ['y'], sep = ",")
dataset.head()

,x1,x2,x3,x4,x5,x6,y
0,date,Temperature,Humidity,Light,CO2,HumidityRatio,Occupancy
1,2015-02-02 14:19:00,23.7,26.272,585.2,749.2,0.0047641630241641,1
2,2015-02-02 14:19:59,23.718,26.29,578.4,760.4,0.0047726609921251,1
3,2015-02-02 14:21:00,23.73,26.23,572.666666666667,769.666666666667,0.0047651525524654,1
4,2015-02-02 14:22:00,23.7225,26.125,493.75,774.75,0.0047437733559968,1


In [151]:
#pick light,co2,y
dataset_light_co2 = dataset.iloc[1:,3:5]
dataset_light_co2['y'] = dataset.iloc[1:,6:7]
dataset_light_co2.columns = ['x1','x2','y']
dataset_light_co2['x1'] = dataset_light_co2['x1'].apply(lambda x : float(x))
dataset_light_co2['x2'] = dataset_light_co2['x2'].apply(lambda x : float(x))
dataset_light_co2['y'] = dataset_light_co2['y'].apply(lambda x : int(x))
dataset_light_co2

,x1,x2,y
1,585.200000,749.200000,1
2,578.400000,760.400000,1
3,572.666667,769.666667,1
4,493.750000,774.750000,1
5,488.600000,779.000000,1
...,...,...,...
20556,429.750000,1505.250000,1
20557,423.500000,1514.500000,1
20558,423.500000,1521.500000,1
20559,418.750000,1632.000000,1


In [152]:
minmax = preprocessing.MinMaxScaler()
dataset_light_co2 = minmax.fit_transform(dataset_light_co2)
dataset_light_co2 = pd.DataFrame(dataset_light_co2)
dataset_light_co2.columns = ['x1','x2','y']
dataset_light_co2

,x1,x2,y
0,0.344793,0.202224,1.0
1,0.340787,0.208956,1.0
2,0.337409,0.214525,1.0
3,0.290912,0.217581,1.0
4,0.287877,0.220135,1.0
...,...,...,...
20555,0.253204,0.656649,1.0
20556,0.249521,0.662209,1.0
20557,0.249521,0.666416,1.0
20558,0.246723,0.732832,1.0


In [153]:
subset = dataset_light_co2.sample(frac=0.2,replace=False,random_state=43)
dataset_light_co2 = dataset_light_co2.append(subset)
dataset_light_co2 = dataset_light_co2.drop_duplicates(subset=['x1','x2','y'],keep=False)

subsample_x = subset.iloc[:,:-1].values
subsample_y = subset.iloc[:,-1].values

#using in Eculidean dist
error_a = []
error_b = []
for i in range(0,21) :
    error_a.append(i)
    error_b.append(20-i)

#using in cosine dist
error_a_cos = []
error_b_cos = []
for i in range(0,25):
    error_a_cos.append(i)
    error_b_cos.append(25-i)

print(error_a,error_b)
print(dataset_light_co2.info(),subset.info())

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20] [20, 19, 18, 17, 16, 15, 14, 13, 12, 11, 10, 9, 8, 7, 6, 5, 4, 3, 2, 1, 0]
<class 'pandas.core.frame.DataFrame'>
Int64Index: 6757 entries, 0 to 20559
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   x1      6757 non-null   float64
 1   x2      6757 non-null   float64
 2   y       6757 non-null   float64
dtypes: float64(3)
memory usage: 211.2 KB
<class 'pandas.core.frame.DataFrame'>
Int64Index: 4112 entries, 8854 to 4673
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   x1      4112 non-null   float64
 1   x2      4112 non-null   float64
 2   y       4112 non-null   float64
dtypes: float64(3)
memory usage: 128.5 KB
None None


In [154]:
importanceReg,RegCoef = featureImportance_linearRegression(subsample_x,subsample_y)
model = KNeighborsClassifier(n_neighbors=5,algorithm="brute")  
model.fit(subsample_x,subsample_y)  
importancePermu,PermuCoef = featureImportance_permutation(model,subsample_x,subsample_y)
importanceDTree,DTreeCoef = featureImportance_DTree(subsample_x,subsample_y)
print("RegCoef : " +str(RegCoef))
print("importanceReg : "+str(importanceReg))
print("PermuCoef : "+str(PermuCoef))
print("importancePermu : "+str(importancePermu))
print("DTreeCoef : " +str(DTreeCoef))
print("importanceDTree : "+str(importanceDTree))

RegCoef : [2.910980313625912, 0.256307040639797]
importanceReg : [0.919076796017702, 0.08092320398229802]
PermuCoef : [0.3004377431906615, 0.05992217898832686]
importancePermu : [0.8337157511135106, 0.1662842488864894]
DTreeCoef : [0.960058786669445, 0.03994121333055508]
importanceDTree : [0.960058786669445, 0.03994121333055508]


In [155]:
#BE for bounded err
#iter for iteration or not
#neg for negative importance or not
#importance for two feature importance ratio
#df for dataframe
#dist 1 for Eculidean dist,2 for cosine dist

def df_2feature_be_generates(BE,iter=1,neg=0,importance=importanceReg,df=dataset_light_co2,dist=1):
    temp = df.copy()
    df_2feature_be = pd.DataFrame.to_numpy(dataset_light_co2)
    e1 = 0
    e2 = 0

    if dist == 1 :#Eculidean distance
        if iter == 1 and neg == 0:
            e1 = temp['x1'].mean()*BE*(error_a[iter]/(error_a[iter]+error_b[iter]))
            e2 = temp['x2'].mean()*BE*(error_b[iter]/(error_a[iter]+error_b[iter]))
        elif iter == 0 and neg == 1:
            e1 = temp['x1'].mean()*BE*importance[0]
            e2 = temp['x2'].mean()*BE*importance[1]
        elif iter == 0 and neg == 0:
            e1 = temp['x1'].mean()*BE*importance[1]
            e2 = temp['x2'].mean()*BE*importance[0]
        
        for idx,cell in temp['x1'].iteritems():
            temp['x1'][idx] += e1
        temp = temp.to_numpy()
        df_2feature_be = np.vstack((df_2feature_be,temp))
        temp = dataset_light_co2.copy()

        for idx,cell in temp['x1'].iteritems():
            temp['x1'][idx] = temp['x1'][idx] - e1
        temp = temp.to_numpy()
        df_2feature_be = np.vstack((df_2feature_be,temp))
        temp = dataset_light_co2.copy()

        for idx,cell in temp['x2'].iteritems():
            temp['x2'][idx] += e2
        temp = temp.to_numpy()
        df_2feature_be = np.vstack((df_2feature_be,temp))
        temp = dataset_light_co2.copy()

        for idx,cell in temp['x2'].iteritems():
            temp['x2'][idx] = temp['x2'][idx] - e2
        temp = temp.to_numpy()
        df_2feature_be = np.vstack((df_2feature_be,temp))
        temp = dataset_light_co2.copy()

        df_2feature_be = pd.DataFrame(df_2feature_be)
        
    elif dist == 2 :#cosine dist
        
        if iter == 1 and neg == 0:
            e1 = temp['x1'].mean()*BE*math.cos((error_a_cos[iter]/(error_a_cos[iter]+error_b_cos[iter])))
            e2 = temp['x2'].mean()*BE*math.sin((error_b_cos[iter]/(error_a_cos[iter]+error_b_cos[iter])))
        elif iter == 0 and neg == 1:
            e1 = temp['x1'].mean()*BE*importance[0]
            e2 = temp['x2'].mean()*BE*importance[1]
        elif iter == 0 and neg == 0:
            e1 = temp['x1'].mean()*BE*importance[1]
            e2 = temp['x2'].mean()*BE*importance[0]

        for idx,cell in temp['x1'].iteritems():
            temp['x1'][idx] += e1
        temp = temp.to_numpy()
        df_2feature_be = np.vstack((df_2feature_be,temp))
        temp = dataset_light_co2.copy()

        for idx,cell in temp['x2'].iteritems():
            temp['x2'][idx] += e2
        temp = temp.to_numpy()
        df_2feature_be = np.vstack((df_2feature_be,temp))
        temp = dataset_light_co2.copy()
        df_2feature_be = pd.DataFrame(df_2feature_be)
    return df_2feature_be


eculidean dist

In [156]:
#using importanceReg as error allocation rate to predict by svm
BE = 0.2
df_2feature_be = df_2feature_be_generates(BE,-1,0,importanceReg)
x_be = df_2feature_be.iloc[:, :-1].values
y_be = df_2feature_be.iloc[:,-1].values
X_train,X_test,y_train, y_test = train_test_split(x_be, y_be, test_size=0.2,random_state=1)
clf_BE_SVC_importanceReg = SVC(kernel='linear',C=1,gamma='auto')
clf_BE_SVC_importanceReg.fit(X_train, y_train)
y_BE_importanceReg_svm_predict = clf_BE_SVC_importanceReg.predict(X_test)
result_BE_train = clf_BE_SVC_importanceReg.score(X_train, y_train)
result_BE_test = clf_BE_SVC_importanceReg.score(X_test, y_test)
subset_raw = clf_BE_SVC_importanceReg.score(subsample_x, subsample_y)
print('BESVM Accuracy (on training) = with x1 bounded error ('+str(BE)+'%*'+str(importanceReg[1])+'),x2 with error ('+str(BE)+'%*'+str(importanceReg[0])+') : '+str(result_BE_train))
print('BESVM Accuracy (on testing) = with x1 bounded error ('+str(BE)+'%*'+str(importanceReg[1])+'),x2 with error ('+str(BE)+'%*'+str(importanceReg[0])+') : '+str(result_BE_test))
print('BESVM Accuracy (on subset_raw) = with no bounded error,x2 with no error '+str(subset_raw))


BESVM Accuracy (on training) = with x1 bounded error (0.2%*0.08092320398229802),x2 with error (0.2%*0.919076796017702) : 0.9734719550096197
BESVM Accuracy (on testing) = with x1 bounded error (0.2%*0.08092320398229802),x2 with error (0.2%*0.919076796017702) : 0.9743969217108184
BESVM Accuracy (on subset_raw) = with no bounded error,x2 with no error 0.9878404669260701


In [157]:
#using importancePermu as error allocation rate to predict by svm
df_2feature_be = df_2feature_be_generates(BE,-1,0,importancePermu)
x_be = df_2feature_be.iloc[:, :-1].values
y_be = df_2feature_be.iloc[:,-1].values
X_train,X_test,y_train, y_test = train_test_split(x_be, y_be, test_size=0.2,random_state=1)
clf_BE_SVC_importancePermu = SVC(kernel='linear',C=1,gamma='auto')
clf_BE_SVC_importancePermu.fit(X_train, y_train)
y_BE_importancePermu_svm_predict = clf_BE_SVC_importancePermu.predict(X_test)
result_BE_train = clf_BE_SVC_importancePermu.score(X_train, y_train)
result_BE_test = clf_BE_SVC_importancePermu.score(X_test, y_test)
subset_raw = clf_BE_SVC_importancePermu.score(subsample_x, subsample_y)
print('BESVM Accuracy (on training) = with x1 bounded error ('+str(BE)+'%*'+str(importancePermu[1])+'),x2 with error ('+str(BE)+'%*'+str(importancePermu[0])+') : '+str(result_BE_train))
print('BESVM Accuracy (on testing) = with x1 bounded error ('+str(BE)+'%*'+str(importancePermu[1])+'),x2 with error ('+str(BE)+'%*'+str(importancePermu[0])+') : '+str(result_BE_test))
print('BESVM Accuracy (on subset_raw) = with no bounded error,x2 with no error '+str(subset_raw))



BESVM Accuracy (on training) = with x1 bounded error (0.2%*0.1662842488864894),x2 with error (0.2%*0.8337157511135106) : 0.9734719550096197
BESVM Accuracy (on testing) = with x1 bounded error (0.2%*0.1662842488864894),x2 with error (0.2%*0.8337157511135106) : 0.9743969217108184
BESVM Accuracy (on subset_raw) = with no bounded error,x2 with no error 0.9878404669260701


In [158]:
#using importanceDTree as error allocation rate to predict by svm
df_2feature_be = df_2feature_be_generates(BE,-1,0,importanceDTree)
x_be = df_2feature_be.iloc[:, :-1].values
y_be = df_2feature_be.iloc[:,-1].values
X_train,X_test,y_train, y_test = train_test_split(x_be, y_be, test_size=0.2,random_state=1)
clf_BE_SVC_importanceDTree = SVC(kernel='linear',C=1,gamma='auto')
clf_BE_SVC_importanceDTree.fit(X_train, y_train)
y_BE_importanceDTree_svm_predict = clf_BE_SVC_importanceDTree.predict(X_test)
result_BE_train = clf_BE_SVC_importanceDTree.score(X_train, y_train)
result_BE_test = clf_BE_SVC_importanceDTree.score(X_test, y_test)
subset_raw = clf_BE_SVC_importanceDTree.score(subsample_x, subsample_y)
print('BESVM Accuracy (on training) = with x1 bounded error ('+str(BE)+'%*'+str(importanceDTree[1])+'),x2 with error ('+str(BE)+'%*'+str(importanceDTree[0])+') : '+str(result_BE_train))
print('BESVM Accuracy (on testing) = with x1 bounded error ('+str(BE)+'%*'+str(importanceDTree[1])+'),x2 with error ('+str(BE)+'%*'+str(importanceDTree[0])+') : '+str(result_BE_test))
print('BESVM Accuracy (on subset_raw) = with no bounded error,x2 with no error '+str(subset_raw))



BESVM Accuracy (on training) = with x1 bounded error (0.2%*0.03994121333055508),x2 with error (0.2%*0.960058786669445) : 0.9734719550096197
BESVM Accuracy (on testing) = with x1 bounded error (0.2%*0.03994121333055508),x2 with error (0.2%*0.960058786669445) : 0.9743969217108184
BESVM Accuracy (on subset_raw) = with no bounded error,x2 with no error 0.9878404669260701


(20%*0.5,20*0.5) could be viewed as initial err distribution

In [159]:
#brute force allocation
#brute force from 0-20 using svm
BE = 0.2
for i in range(21) :
    df_2feature_be = df_2feature_be_generates(BE,i,0)
    x_be = df_2feature_be.iloc[:, :-1].values
    y_be = df_2feature_be.iloc[:,-1].values
    X_train,X_test,y_train, y_test = train_test_split(x_be, y_be, test_size=0.2,random_state=1)
    clf_BE = SVC(kernel='linear',C=1,gamma='auto')
    clf_BE.fit(X_train, y_train)
    y_BE_predict = clf_BE.predict(X_test)
    result_BE_train = clf_BE.score(X_train, y_train)
    result_BE_test = clf_BE.score(X_test, y_test)
    subset_raw = clf_BE.score(subsample_x, subsample_y)
    print('BESVM Accuracy (on training) = with x1 bounded error ('+str(BE)+'%*'+str((error_a[i]/(error_a[i]+error_b[i])))+'),x2 with error ('+str(BE)+'%*'+str((error_b[i]/(error_a[i]+error_b[i])))+') : '+str(result_BE_train))
    print('BESVM Accuracy (on testing) = with x1 bounded error ('+str(BE)+'%*'+str((error_a[i]/(error_a[i]+error_b[i])))+'),x2 with error ('+str(BE)+'%*'+str((error_b[i]/(error_a[i]+error_b[i])))+') : '+str(result_BE_test))
    print('BESVM Accuracy (on subset_raw) = with no bounded error,x2 with no error '+str(subset_raw))
    

BESVM Accuracy (on training) = with x1 bounded error (0.2%*0.0),x2 with error (0.2%*1.0) : 0.9735089536776677
BESVM Accuracy (on testing) = with x1 bounded error (0.2%*0.0),x2 with error (0.2%*1.0) : 0.973952937694243
BESVM Accuracy (on subset_raw) = with no bounded error,x2 with no error 0.9878404669260701
BESVM Accuracy (on training) = with x1 bounded error (0.2%*0.05),x2 with error (0.2%*0.95) : 0.9736199496818114
BESVM Accuracy (on testing) = with x1 bounded error (0.2%*0.05),x2 with error (0.2%*0.95) : 0.973952937694243
BESVM Accuracy (on subset_raw) = with no bounded error,x2 with no error 0.9878404669260701
BESVM Accuracy (on training) = with x1 bounded error (0.2%*0.1),x2 with error (0.2%*0.9) : 0.9734719550096197
BESVM Accuracy (on testing) = with x1 bounded error (0.2%*0.1),x2 with error (0.2%*0.9) : 0.9743969217108184
BESVM Accuracy (on subset_raw) = with no bounded error,x2 with no error 0.9878404669260701
BESVM Accuracy (on training) = with x1 bounded error (0.2%*0.15),x2 

In [160]:
#using importanceReg as error allocation rate to predict by knn
df_2feature_be = df_2feature_be_generates(BE,-1,0,importanceReg)
x_be = df_2feature_be.iloc[:, :-1].values
y_be = df_2feature_be.iloc[:,-1].values
X_train,X_test,y_train, y_test = train_test_split(x_be, y_be, test_size=0.2,random_state=1)
KNN_BE_importanceReg = KNeighborsClassifier()
KNN_BE_importanceReg.fit(X_train, y_train)
y_BE_importanceReg_knn_predict = KNN_BE_importanceReg.predict(X_test)
result_BE_train = KNN_BE_importanceReg.score(X_train, y_train)
result_BE_test = KNN_BE_importanceReg.score(X_test, y_test)
subset_raw = KNN_BE_importanceReg.score(subsample_x, subsample_y)
print('BEKNN Accuracy (on training) = with x1 bounded error ('+str(BE)+'%*'+str(importanceReg[1])+'),x2 with error ('+str(BE)+'%*'+str(importanceReg[0])+') : '+str(result_BE_train))
print('BEKNN Accuracy (on testing) = with x1 bounded error ('+str(BE)+'%*'+str(importanceReg[1])+'),x2 with error ('+str(BE)+'%*'+str(importanceReg[0])+') : '+str(result_BE_test))
print('BEKNN Accuracy (on subset_raw) = with no bounded error,x2 with no error '+str(subset_raw))


BEKNN Accuracy (on training) = with x1 bounded error (0.2%*0.08092320398229802),x2 with error (0.2%*0.919076796017702) : 0.9991490306348971
BEKNN Accuracy (on testing) = with x1 bounded error (0.2%*0.08092320398229802),x2 with error (0.2%*0.919076796017702) : 0.9945241971289034
BEKNN Accuracy (on subset_raw) = with no bounded error,x2 with no error 0.982976653696498


In [162]:
#using importancePermu as error allocation rate to predict by knn
df_2feature_be = df_2feature_be_generates(BE,-1,0,importancePermu)
x_be = df_2feature_be.iloc[:, :-1].values
y_be = df_2feature_be.iloc[:,-1].values
X_train,X_test,y_train, y_test = train_test_split(x_be, y_be, test_size=0.2,random_state=1)
KNN_BE_importancePermu = KNeighborsClassifier()
KNN_BE_importancePermu.fit(X_train, y_train)
y_BE_importancePermu_knn_predict = KNN_BE_importancePermu.predict(X_test)
result_BE_train = KNN_BE_importancePermu.score(X_train, y_train)
result_BE_test = KNN_BE_importancePermu.score(X_test, y_test)
subset_raw = KNN_BE_importancePermu.score(subsample_x, subsample_y)
print('BEKNN Accuracy (on training) = with x1 bounded error ('+str(BE)+'%*'+str(importancePermu[1])+'),x2 with error ('+str(BE)+'%*'+str(importancePermu[0])+') : '+str(result_BE_train))
print('BEKNN Accuracy (on testing) = with x1 bounded error ('+str(BE)+'%*'+str(importancePermu[1])+'),x2 with error ('+str(BE)+'%*'+str(importancePermu[0])+') : '+str(result_BE_test))
print('BEKNN Accuracy (on subset_raw) = with no bounded error,x2 with no error '+str(subset_raw))


BEKNN Accuracy (on training) = with x1 bounded error (0.2%*0.1662842488864894),x2 with error (0.2%*0.8337157511135106) : 0.9991490306348971
BEKNN Accuracy (on testing) = with x1 bounded error (0.2%*0.1662842488864894),x2 with error (0.2%*0.8337157511135106) : 0.9945241971289034
BEKNN Accuracy (on subset_raw) = with no bounded error,x2 with no error 0.982976653696498


In [163]:
#using importanceDTree as error allocation rate to predict by knn
df_2feature_be = df_2feature_be_generates(BE,-1,0,importanceDTree)
x_be = df_2feature_be.iloc[:, :-1].values
y_be = df_2feature_be.iloc[:,-1].values
X_train,X_test,y_train, y_test = train_test_split(x_be, y_be, test_size=0.2,random_state=1)
KNN_BE_importanceDTree = KNeighborsClassifier()
KNN_BE_importanceDTree.fit(X_train, y_train)
y_BE_importanceDTree_knn_predict = KNN_BE_importanceDTree.predict(X_test)
result_BE_train = KNN_BE_importanceDTree.score(X_train, y_train)
result_BE_test = KNN_BE_importanceDTree.score(X_test, y_test)
subset_raw = KNN_BE_importanceDTree.score(subsample_x, subsample_y)
print('BEKNN Accuracy (on training) = with x1 bounded error ('+str(BE)+'%*'+str(importanceDTree[1])+'),x2 with error ('+str(BE)+'%*'+str(importanceDTree[0])+') : '+str(result_BE_train))
print('BEKNN Accuracy (on testing) = with x1 bounded error ('+str(BE)+'%*'+str(importanceDTree[1])+'),x2 with error ('+str(BE)+'%*'+str(importanceDTree[0])+') : '+str(result_BE_test))
print('BEKNN Accuracy (on subset_raw) = with no bounded error,x2 with no error '+str(subset_raw))


BEKNN Accuracy (on training) = with x1 bounded error (0.2%*0.03994121333055508),x2 with error (0.2%*0.960058786669445) : 0.9991490306348971
BEKNN Accuracy (on testing) = with x1 bounded error (0.2%*0.03994121333055508),x2 with error (0.2%*0.960058786669445) : 0.9945241971289034
BEKNN Accuracy (on subset_raw) = with no bounded error,x2 with no error 0.982976653696498


In [164]:
#using knn with brute force from 0-20
for i in range(21):
    df_2feature_be = df_2feature_be_generates(BE,i,0)
    x_be = df_2feature_be.iloc[:, :-1].values
    y_be = df_2feature_be.iloc[:,-1].values
    X_train,X_test,y_train, y_test = train_test_split(x_be, y_be, test_size=0.2,random_state=1)
    KNN_BE = KNeighborsClassifier()
    KNN_BE.fit(X_train,y_train)
    y_BE_predict = KNN_BE.predict(X_test)
    result_BE_train = KNN_BE.score(X_train, y_train)
    result_BE_test = KNN_BE.score(X_test, y_test)
    subset_raw = KNN_BE.score(subsample_x, subsample_y)
    
    print('BEKNN Accuracy (on training) = with x1 bounded error ('+str(BE)+'%*'+str((error_a[i]/(error_a[i]+error_b[i])))+'),x2 with error ('+str(BE)+'%*'+str((error_b[i]/(error_a[i]+error_b[i])))+') : '+str(result_BE_train))
    print('BEKNN Accuracy (on testing) = with x1 bounded error ('+str(BE)+'%*'+str((error_a[i]/(error_a[i]+error_b[i])))+'),x2 with error ('+str(BE)+'%*'+str((error_b[i]/(error_a[i]+error_b[i])))+') : '+str(result_BE_test))
    print('BEKNN Accuracy (on subset_raw) = with no bounded error,x2 with no error '+str(subset_raw))
    

BEKNN Accuracy (on training) = with x1 bounded error (0.2%*0.0),x2 with error (0.2%*1.0) : 0.9798357259138671
BEKNN Accuracy (on testing) = with x1 bounded error (0.2%*0.0),x2 with error (0.2%*1.0) : 0.9748409057273938
BEKNN Accuracy (on subset_raw) = with no bounded error,x2 with no error 0.9878404669260701
BEKNN Accuracy (on training) = with x1 bounded error (0.2%*0.05),x2 with error (0.2%*0.95) : 0.9815746633121207
BEKNN Accuracy (on testing) = with x1 bounded error (0.2%*0.05),x2 with error (0.2%*0.95) : 0.9745449163830102
BEKNN Accuracy (on subset_raw) = with no bounded error,x2 with no error 0.9885700389105059
BEKNN Accuracy (on training) = with x1 bounded error (0.2%*0.1),x2 with error (0.2%*0.9) : 0.9991490306348971
BEKNN Accuracy (on testing) = with x1 bounded error (0.2%*0.1),x2 with error (0.2%*0.9) : 0.9945241971289034
BEKNN Accuracy (on subset_raw) = with no bounded error,x2 with no error 0.982976653696498
BEKNN Accuracy (on training) = with x1 bounded error (0.2%*0.15),x2

In [165]:
#importanceReg,KMeans
df_2feature_be = df_2feature_be_generates(BE,-1,0,importanceReg)
x_be = df_2feature_be.iloc[:, :-1].values
#x_be = preprocessing.scale(x_be)#標準化
y_be = df_2feature_be.iloc[:,-1].values
X_train,X_test,y_train, y_test = train_test_split(x_be, y_be, test_size=0.2,random_state=1)
KM_BE_importanceReg=KMeans(n_clusters=1,init='random',random_state=5)
KM_BE_importanceReg.fit(X_train)
y_BE_importanceReg_KMeans_predict = KM_BE_importanceReg.predict(X_train)
y_BE_predict_test = KM_BE_importanceReg.predict(X_test)
#subsample_x = preprocessing.scale(subsample_x)
y_subset = KM_BE_importanceReg.predict(subsample_x)
result_BE_train = 1 - np.sum(y_BE_predict!=y_train)/len(y_train)
result_BE_test = 1 - np.sum(y_BE_predict_test!=y_train)/len(y_test)
subset_raw = 1 - np.sum(subsample_y!=y_subset)/len(subsample_y)
    
    
print('BEKM Accuracy (on training) = with x1 bounded error ('+str(BE)+'%*'+str(importanceReg[1])+'),x2 with error ('+str(BE)+'%*'+str(importanceReg[0])+') : '+str(result_BE_train))
print('BEKM Accuracy (on testing) = with x1 bounded error ('+str(BE)+'%*'+str(importanceReg[1])+'),x2 with error ('+str(BE)+'%*'+str(importanceReg[0])+') : '+str(result_BE_test))
print('BEKM Accuracy (on subset_raw) = with no bounded error,x2 with no error '+str(subset_raw))

BEKM Accuracy (on training) = with x1 bounded error (0.2%*0.08092320398229802),x2 with error (0.2%*0.919076796017702) : 0.999963001331952
BEKM Accuracy (on testing) = with x1 bounded error (0.2%*0.08092320398229802),x2 with error (0.2%*0.919076796017702) : 0.9998520053278082
BEKM Accuracy (on subset_raw) = with no bounded error,x2 with no error 0.7665369649805447


d:\BESVM\BESVM\.venv\lib\site-packages\ipykernel_launcher.py:13: DeprecationWarning: elementwise comparison failed; this will raise an error in the future.
  del sys.path[0]
d:\BESVM\BESVM\.venv\lib\site-packages\ipykernel_launcher.py:14: DeprecationWarning: elementwise comparison failed; this will raise an error in the future.
  


In [166]:
#importancePermu,KMeans
df_2feature_be = df_2feature_be_generates(BE,-1,0,importancePermu)
x_be = df_2feature_be.iloc[:, :-1].values
#x_be = preprocessing.scale(x_be)#標準化
y_be = df_2feature_be.iloc[:,-1].values
X_train,X_test,y_train, y_test = train_test_split(x_be, y_be, test_size=0.2,random_state=1)
KM_BE_importancePermu=KMeans(n_clusters=1,init='random',random_state=5)
KM_BE_importancePermu.fit(X_train)
y_BE_importancePermu_KMeans__predict = KM_BE_importancePermu.predict(X_train)
y_BE_predict_test = KM_BE_importancePermu.predict(X_test)
subsample_x = preprocessing.scale(subsample_x)
y_subset = KM_BE_importancePermu.predict(subsample_x)
result_BE_train = 1 - np.sum(y_BE_predict!=y_train)/len(y_train)
result_BE_test = 1 - np.sum(y_BE_predict_test!=y_train)/len(y_test)
subset_raw = 1 - np.sum(subsample_y!=y_subset)/len(subsample_y)
    
    
print('BEKM Accuracy (on training) = with x1 bounded error ('+str(BE)+'%*'+str(importancePermu[1])+'),x2 with error ('+str(BE)+'%*'+str(importancePermu[0])+') : '+str(result_BE_train))
print('BEKM Accuracy (on testing) = with x1 bounded error ('+str(BE)+'%*'+str(importancePermu[1])+'),x2 with error ('+str(BE)+'%*'+str(importancePermu[0])+') : '+str(result_BE_test))
print('BEKM Accuracy (on subset_raw) = with no bounded error,x2 with no error '+str(subset_raw))

BEKM Accuracy (on training) = with x1 bounded error (0.2%*0.1662842488864894),x2 with error (0.2%*0.8337157511135106) : 0.999963001331952
BEKM Accuracy (on testing) = with x1 bounded error (0.2%*0.1662842488864894),x2 with error (0.2%*0.8337157511135106) : 0.9998520053278082
BEKM Accuracy (on subset_raw) = with no bounded error,x2 with no error 0.7665369649805447


d:\BESVM\BESVM\.venv\lib\site-packages\ipykernel_launcher.py:13: DeprecationWarning: elementwise comparison failed; this will raise an error in the future.
  del sys.path[0]
d:\BESVM\BESVM\.venv\lib\site-packages\ipykernel_launcher.py:14: DeprecationWarning: elementwise comparison failed; this will raise an error in the future.
  


In [167]:
#importanceDTree,KMeans
df_2feature_be = df_2feature_be_generates(BE,-1,0,importanceDTree)
x_be = df_2feature_be.iloc[:, :-1].values
#x_be = preprocessing.scale(x_be)#標準化
y_be = df_2feature_be.iloc[:,-1].values
X_train,X_test,y_train, y_test = train_test_split(x_be, y_be, test_size=0.2,random_state=1)
KM_BE_importanceDTree=KMeans(n_clusters=1,init='random',random_state=5)
KM_BE_importanceDTree.fit(X_train)
y_BE_importanceDTree_KMeans_predict = KM_BE_importanceDTree.predict(X_train)
y_BE_predict_test = KM_BE_importanceDTree.predict(X_test)
#subsample_x = preprocessing.scale(subsample_x)
y_subset = KM_BE_importanceDTree.predict(subsample_x)
result_BE_train = 1 - np.sum(y_BE_predict!=y_train)/len(y_train)
result_BE_test = 1 - np.sum(y_BE_predict_test!=y_train)/len(y_test)
subset_raw = 1 - np.sum(subsample_y!=y_subset)/len(subsample_y)
    
    
print('BEKM Accuracy (on training) = with x1 bounded error ('+str(BE)+'%*'+str(importanceDTree[1])+'),x2 with error ('+str(BE)+'%*'+str(importanceDTree[0])+') : '+str(result_BE_train))
print('BEKM Accuracy (on testing) = with x1 bounded error ('+str(BE)+'%*'+str(importanceDTree[1])+'),x2 with error ('+str(BE)+'%*'+str(importanceDTree[0])+') : '+str(result_BE_test))
print('BEKM Accuracy (on subset_raw) = with no bounded error,x2 with no error '+str(subset_raw))

BEKM Accuracy (on training) = with x1 bounded error (0.2%*0.03994121333055508),x2 with error (0.2%*0.960058786669445) : 0.999963001331952
BEKM Accuracy (on testing) = with x1 bounded error (0.2%*0.03994121333055508),x2 with error (0.2%*0.960058786669445) : 0.9998520053278082
BEKM Accuracy (on subset_raw) = with no bounded error,x2 with no error 0.7665369649805447


d:\BESVM\BESVM\.venv\lib\site-packages\ipykernel_launcher.py:13: DeprecationWarning: elementwise comparison failed; this will raise an error in the future.
  del sys.path[0]
d:\BESVM\BESVM\.venv\lib\site-packages\ipykernel_launcher.py:14: DeprecationWarning: elementwise comparison failed; this will raise an error in the future.
  


In [168]:
#using kMeans with brute force from 0-20
for i in range(21):
    df_2feature_be = df_2feature_be_generates(BE,i,0)
    x_be = df_2feature_be.iloc[:, :-1].values
    #x_be = preprocessing.scale(x_be)#標準化
    y_be = df_2feature_be.iloc[:,-1].values
    X_train,X_test,y_train, y_test = train_test_split(x_be, y_be, test_size=0.2,random_state=1)
    KM_BE=KMeans(n_clusters=1,init='random',random_state=5)
    KM_BE.fit(X_train)
    y_BE_predict = KM_BE.predict(X_train)
    y_BE_predict_test = KM_BE.predict(X_test)
    subsample_x = preprocessing.scale(subsample_x)
    y_subset = KM_BE.predict(subsample_x)
    result_BE_train = 1 - np.sum(y_BE_predict!=y_train)/len(y_train)
    #print(score)
    result_BE_test = 1 - np.sum(y_BE_predict_test!=y_train)/len(y_test)
    subset_raw = 1 - np.sum(subsample_y!=y_subset)/len(subsample_y)
    #print(score)
    #result_BE_train = KM_BE.score(X_train, y_train)
    #result_BE_test = KM_BE.score(X_test, y_test)
    #subset_raw = KM_BE.score(subsample_x, subsample_y)
    
    
    print('BEKM Accuracy (on training) = with x1 bounded error ('+str(BE)+'%*'+str((error_a[i]/(error_a[i]+error_b[i])))+'),x2 with error ('+str(BE)+'%*'+str((error_b[i]/(error_a[i]+error_b[i])))+') : '+str(result_BE_train))
    print('BEKM Accuracy (on testing) = with x1 bounded error ('+str(BE)+'%*'+str((error_a[i]/(error_a[i]+error_b[i])))+'),x2 with error ('+str(BE)+'%*'+str((error_b[i]/(error_a[i]+error_b[i])))+') : '+str(result_BE_test))
    print('BEKM Accuracy (on subset_raw) = with no bounded error,x2 with no error '+str(subset_raw))
    

d:\BESVM\BESVM\.venv\lib\site-packages\ipykernel_launcher.py:16: DeprecationWarning: elementwise comparison failed; this will raise an error in the future.
  


BEKM Accuracy (on training) = with x1 bounded error (0.2%*0.0),x2 with error (0.2%*1.0) : 0.4710670415865029
BEKM Accuracy (on testing) = with x1 bounded error (0.2%*0.0),x2 with error (0.2%*1.0) : 0.9998520053278082
BEKM Accuracy (on subset_raw) = with no bounded error,x2 with no error 0.7665369649805447


d:\BESVM\BESVM\.venv\lib\site-packages\ipykernel_launcher.py:16: DeprecationWarning: elementwise comparison failed; this will raise an error in the future.
  


BEKM Accuracy (on training) = with x1 bounded error (0.2%*0.05),x2 with error (0.2%*0.95) : 0.4710670415865029
BEKM Accuracy (on testing) = with x1 bounded error (0.2%*0.05),x2 with error (0.2%*0.95) : 0.9998520053278082
BEKM Accuracy (on subset_raw) = with no bounded error,x2 with no error 0.7665369649805447


d:\BESVM\BESVM\.venv\lib\site-packages\ipykernel_launcher.py:16: DeprecationWarning: elementwise comparison failed; this will raise an error in the future.
  


...

d:\BESVM\BESVM\.venv\lib\site-packages\ipykernel_launcher.py:16: DeprecationWarning: elementwise comparison failed; this will raise an error in the future.
  


BEKM Accuracy (on training) = with x1 bounded error (0.2%*0.9),x2 with error (0.2%*0.1) : 0.4710670415865029
BEKM Accuracy (on testing) = with x1 bounded error (0.2%*0.9),x2 with error (0.2%*0.1) : 0.9998520053278082
BEKM Accuracy (on subset_raw) = with no bounded error,x2 with no error 0.7665369649805447


d:\BESVM\BESVM\.venv\lib\site-packages\ipykernel_launcher.py:16: DeprecationWarning: elementwise comparison failed; this will raise an error in the future.
  


BEKM Accuracy (on training) = with x1 bounded error (0.2%*0.95),x2 with error (0.2%*0.05) : 0.4710670415865029
BEKM Accuracy (on testing) = with x1 bounded error (0.2%*0.95),x2 with error (0.2%*0.05) : 0.9998520053278082
BEKM Accuracy (on subset_raw) = with no bounded error,x2 with no error 0.7665369649805447
BEKM Accuracy (on training) = with x1 bounded error (0.2%*1.0),x2 with error (0.2%*0.0) : 0.4710670415865029
BEKM Accuracy (on testing) = with x1 bounded error (0.2%*1.0),x2 with error (0.2%*0.0) : 0.9998520053278082
BEKM Accuracy (on subset_raw) = with no bounded error,x2 with no error 0.7665369649805447


d:\BESVM\BESVM\.venv\lib\site-packages\ipykernel_launcher.py:16: DeprecationWarning: elementwise comparison failed; this will raise an error in the future.
  


implement the ML trust model,test ML models which is trained above

In [169]:
#take err distribution 20%*0.5,20%*0.5 as ML model in edge and train knn
importanceEdge = [0.5,0.5]
#using importanceReg as error allocation rate to predict by svm
BE = 0.2
df_2feature_be = df_2feature_be_generates(BE,-1,0,importanceEdge)
x_be = df_2feature_be.iloc[:, :-1].values
y_be = df_2feature_be.iloc[:,-1].values
X_train,X_test,y_train, y_test = train_test_split(x_be, y_be, test_size=0.2,random_state=1)
clf_BE_SVC_importanceEdge = SVC(kernel='linear',C=1,gamma='auto')
clf_BE_SVC_importanceEdge.fit(X_train, y_train)
y_BE_importanceEdge_knn_predict = clf_BE_SVC_importanceEdge.predict(X_test)
result_BE_train = clf_BE_SVC_importanceEdge.score(X_train, y_train)
result_BE_test = clf_BE_SVC_importanceEdge.score(X_test, y_test)

print(X_test.shape,y_BE_predict.shape)
print(X_train.shape)

## preprocess data for knn from ml models
KNN_Xtest_predictY = pd.DataFrame(X_test)
y = pd.DataFrame(y_BE_predict)
KNN_Xtest_predictY['y'] = y
KNN_Xtest_predictY.columns = ['x1','x2','y']
print(KNN_Xtest_predictY.info())       







(6757, 2) (27028,)
(27028, 2)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6757 entries, 0 to 6756
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   x1      6757 non-null   float64
 1   x2      6757 non-null   float64
 2   y       6757 non-null   int32  
dtypes: float64(2), int32(1)
memory usage: 132.1 KB
None


In [170]:
X_fitKNN = KNN_Xtest_predictY.iloc[:,:-1]
y_fitKNN = KNN_Xtest_predictY.iloc[:,-1]
KNN_trust_importanceEdge = KNeighborsClassifier(n_neighbors=3,weights='uniform',algorithm='brute',p=2)

X_fitKNN = np.array(X_fitKNN)
y_fitKNN = np.array(y_fitKNN)

KNN_trust_importanceEdge.fit(X_fitKNN,y_fitKNN)

# x is used for ML training above are here as predict data for KNN
y_KNN_trust_predict =  KNN_trust_importanceEdge.predict(X_train)
print(np.size(y_KNN_trust_predict),np.size(y_train))

27028 27028


In [171]:
#trust computing data preprocessing
data_trust_KNN = pd.DataFrame(X_train)
data_trust_KNN['y'] = y_KNN_trust_predict
data_trust_KNN.columns = ['x1','x2','y']

data_trust = pd.DataFrame(X_train)
data_trust['y'] = y_train
data_trust.columns = ['x1','x2','y']

print(data_trust_KNN.info(),data_trust.info())
#print(data_trust_KNN)
print(data_trust) 

## trust computing




<class 'pandas.core.frame.DataFrame'>
RangeIndex: 27028 entries, 0 to 27027
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   x1      27028 non-null  float64
 1   x2      27028 non-null  float64
 2   y       27028 non-null  int32  
dtypes: float64(2), int32(1)
memory usage: 528.0 KB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 27028 entries, 0 to 27027
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   x1      27028 non-null  float64
 1   x2      27028 non-null  float64
 2   y       27028 non-null  float64
dtypes: float64(3)
memory usage: 633.6 KB
None None
             x1        x2    y
0      0.094270  0.166041  0.0
1      0.273383  0.451841  1.0
2      0.215938  0.130428  0.0
3      0.267492  0.320661  1.0
4      0.000000  0.029842  0.0
...         ...       ...  ...
27023  0.266902  0.184373  1.0
27024  0.000000  0.224543  0.0
27025  0.182648  0.132081  0.0


In [172]:
y_train = np.array(y_train)
index = np.arange(0,27028)
equal = []
diff = []
for i in range(len(y_train)):
    if y_train[i] == y_KNN_trust_predict[i]:
        equal.append(i)
    else:
        diff.append(i)

print(len(diff))

14296


In [173]:
X_train[diff[1]]

array([0.26749153, 0.32066116])

In [174]:
dist = KNN_trust_importanceEdge.kneighbors(X_train,n_neighbors=3)
dist

(array([[0.00569549, 0.00659161, 0.01347347],
        [0.00015026, 0.00150263, 0.0031078 ],
        [0.00435155, 0.00435155, 0.00747784],
        ...,
        [0.00137384, 0.00170298, 0.00203664],
        [0.        , 0.        , 0.00020035],
        [0.00064112, 0.00110193, 0.0012021 ]]),
 array([[ 607,  706, 4467],
        [3302, 6507, 3254],
        [ 667, 5916, 1014],
        ...,
        [2236,  690, 2576],
        [1284, 1626, 3207],
        [3760, 2752,  680]], dtype=int64))

In [175]:
def trustComputing(train_x,train_y,predict_y,dist):
    sum = 0
    for i in range(len(predict_y)):
        if train_y[i] == predict_y[i]:
            if predict_y[i] == 0:
                ## cal
                temp_0=[]
                sum_0=1
                for j in range(3):
                    if train_y[dist[1][i][j]] == 0:
                        temp_0.append(dist[0][i][j])
                        sum_0+=dist[0][i][j]
                sum = sum+(1/(sum_0/(len(temp_0)+1)))
                print(1/(sum_0/(len(temp_0)+1)))
                print("test2")
                temp_0.clear()
            else :
                ## cal
                temp_1=[]
                sum_1=1
                for j in range(3):
                    if train_y[dist[1][i][j]] == 1:
                        temp_1.append(dist[0][i][j])
                        sum_1+=dist[0][i][j]
                sum = sum+(1/(sum_1/(len(temp_1)+1)))
                print(1/(sum_1/(len(temp_1)+1)))
                print("test2")
                temp_1.clear()
        else :
            if predict_y[i] == 1:
                temp_1=[]
                temp_0=[]
                sum_0=1
                sum_1=1

                for j in range(3):
                    if train_y[dist[1][i][j]] == 0 :
                        temp_0.append(dist[0][i][j])
                        sum_0+=dist[0][i][j]
                    else:
                        temp_1.append(dist[0][i][j])
                        sum_1+=dist[0][i][j]
                if(len(temp_0)==0 or len(temp_1)==0):               
                    print(0)
                else:
                    
                    if min(temp_0) < min(temp_1):
                        sum = sum+(1/(sum_1/2+1))
                        print((1/(sum_1/2+1)))
                    else:
                        sum = sum+(1/(sum_0+1))
                        print((1/(sum_0+1)))
                print("test1")
                temp_0.clear()
                temp_1.clear()
            else :
                temp_1=[]
                temp_0=[]
                sum_0=1
                sum_1=1

                for j in range(3):
                    print(train_y[dist[1][i][j]])
                    if train_y[dist[1][i][j]] == 0:
                        temp_0.append(dist[0][i][j])
                        sum_0+=dist[0][i][j]
                    else:
                        temp_1.append(dist[0][i][j])
                        sum_1+=dist[0][i][j]
                if(len(temp_1)==0 or len(temp_0)==0):               
                    print(0)
                else:
                    
                    if min(temp_1) < min(temp_0):
                        sum = sum+(1/(sum_0/2+1))
                        print((1/(sum_0/2+1)))
                    else:
                        sum = sum+(1/(sum_1+1))
                        print((1/(sum_1+1)))
                print("test1")
                temp_0.clear()
                temp_1.clear()
    aver = sum/len(train_y)
    return aver

In [176]:
score = trustComputing(X_train,y_train,y_KNN_trust_predict,dist)
print("score = ",score)

2.9635861209051573
test2
0.0
0.0
0.0
0
test1
1.9913346011060233
test2
1.0
0.0
1.0
0.6661715232953949
test1
2.999278911155845
test2
3.995197358547474
test2
0.0
1.0
1.0
0.4975925618657964
test1
2.0
test2
2.9838589149603956
test2
1.0
1.0
1.0
0
test1
1.0
1.0
0.0
0.4990764143886098
test1
0.0
0.0
1.0
0.4990511794384424
test1
1.993281424697138
test2
1.0
1.0
1.0
0
test1
0.0
0.0
1.0
0.4946584239034084
test1
1.0
test2
1.0
test2
0.0
1.0
1.0
0.49941371144377017
test1
1.0
1.0
1.0
0
test1
1.9850688027508538
test2
2.0
test2
0.0
1.0
0.0
0.4998438543327864
test1
1.995237457815587
test2
1.0
0.0
0.0
0.49999999813735485
test1
3.995729361578548
test2
1.998300413549786
test2
3.980858381935104
test2
2.0
test2
0.0
0.0
1.0
0.4997496871088763
test1
2.0
test2
1.9983984785546247
test2
0.0
0.0
0.0
0
test1
1.0
0.0
0.0
0.665999499624717
test1
1.992718359987309
test2
1.0
1.0
1.0
0
test1
1.0
test2
1.9997996694545574
test2
2.9913099935074676
test2
2.996797013298102
test2
1.0
test2
2.9938189879829236
test2
1.0
1.0
0.0
0

check the accuracy in edge ml
trust computing in step2 (after be ml)
inflate the no error data do ml vs BE distributed data with ml

In [187]:
#combine raw data x and the predicted y by BE model
dataset_light_co2_BEML_y = dataset_light_co2
dataset_light_co2_BEML_y['y'] = y_BE_importanceReg_svm_predict
# train knn
X_fitKNN = dataset_light_co2_BEML_y.iloc[:,:-1]
y_fitKNN = dataset_light_co2_BEML_y.iloc[:,-1]
X_fitKNN = np.array(X_fitKNN)
y_fitKNN = np.array(y_fitKNN)
KNN_trust_BEML = KNeighborsClassifier(n_neighbors=3,weights='uniform',algorithm='brute',p=2)
KNN_trust_BEML.fit(X_fitKNN,y_fitKNN)





KNeighborsClassifier(algorithm='brute', n_neighbors=3)

In [188]:
#
y_BEML_KNN_trust_predict = KNN_trust_BEML.predict(subsample_x)
print(np.size(y_BEML_KNN_trust_predict),np.size(subsample_y))

4112 4112


In [189]:
dist = KNN_trust_BEML.kneighbors(subsample_x,n_neighbors=3)
dist

(array([[0.969713  , 0.97001848, 0.97062963],
        [0.56184482, 0.56184542, 0.56186394],
        [0.94464921, 0.94494871, 0.94554793],
        ...,
        [1.03375166, 1.03407007, 1.03470707],
        [0.84594096, 0.84622373, 0.84678957],
        [1.04256697, 1.04288694, 1.04352706]]),
 array([[2953, 2950, 2951],
        [ 810, 3428, 3429],
        [2953, 2950, 2951],
        ...,
        [2953, 2950, 2951],
        [2953, 2950, 2951],
        [2953, 2950, 2951]], dtype=int64))

In [198]:
print(np.size(dist))

24672


In [192]:
score_importanceReg_svm = trustComputing(subsample_x,subsample_y,y_BEML_KNN_trust_predict,dist)
print("score = ",score_importanceReg_svm)

0.2537653634768424
test1
0.561503761171321
test1
0.25703659311179844
test1
0.25020862870736815
test1
0.2461285544067667
test1
0.0
0.0
1.0
0.3542072021649482
test1
1.0
test2
0.0
0.0
0.0
0
test1
0.2455211680444891
test1
0.2901823350747701
test1
0.28734007409333234
test1
1.3021285796284063
test2
0.7993297389983468
test2
0.24622990917368537
test1
0.26220578677690903
test1
1.0
test2


IndexError: index 6578 is out of bounds for axis 0 with size 4112